# Weighted distance transforms

A distance transform of a domain $\Omega$ (let's say, an image, or an environment) is a function $u$ defined over that domain where for each location $x \in \Omega$ the value $\Phi(x) \geq 0$ represents the distance to a predetermined set of points. These points could be an object in the image, or some 'goal' in the domain. 
As the above sentence suggests, distance transforms are commonly used in image analysis and motion planning.

SciPy implements a distance transform function. It is horribly fast and I'd love to use it, the only problem is the limited scope. It allows only for binary image values: either 'goal' (0) or 'space' (1), essentially solving the equation

$$\left|\left|\nabla \Phi(x)\right|\right|^2 = 1$$

I need a *generalized* or *weighted* distance transform. The image can take all nonnegative values, including $+\infty$.
Zero values still indicate goals, but any other positive value now indicates time (or effort) spent instead of Euclidian distance. The value $\infty$ corresponds to an unaccessible location. 
Let's assume that the speed that can be attained in location $x$ is denoted with some function $u(x)$. I'd like the answer to 

$$\left|\left|\nabla \Phi(x)\right|\right|^2 = u(x)$$

This notebook implements a fast marching algorithm: upwind scheme meets Dijkstra's algorithm.


In [ ]:
import numpy as np
import math
import heapq
import operator

nx,ny = 50,60
all_cells = {(i,j) for i in range(nx) for j in range(ny)}
test_u = np.ones([nx,ny])
circle_loc = np.array([0.5,0.2])

DIR_STRINGS = ["left","up","right","down"]
AXES = ['x','y']
DIRS = np.array([[-1,0],[0,1],[1,0],[0,-1]])
# TODO: Change "potentials" to "distances"
def exists(index):
        """
        Checks whether an index exists an array
        :param index: 2D index tuple
        :param max_index: max index tuple
        :return: true if lower than tuple, false otherwise
        """
        return (0 <= index[0] < nx) and (0 <= index[1] < ny)

def get_new_candidate_cells(new_known_cells,unknown_cells):
    new_candidate_cells = set()
    for cell in new_known_cells:
        for direction in DIRS:
            nb_cell = (cell[0] + direction[0],cell[1] + direction[1])
            if nb_cell in unknown_cells:
                new_candidate_cells.add(nb_cell)
    return new_candidate_cells

def compute_potential(cell,costs,potential):
    # Find the minimal directions along a grid cell.
    # Assume left and below are best, then overwrite with right and up if they are better
    adjacent_potentials = np.ones(4)*np.inf
    pots_from_axis = [0,0] # [x direction, y direction]
    costs_from_axis = [np.inf,np.inf] # 
    for i,dir_s in enumerate(DIR_STRINGS):
        # Direction for which we check the cost
        normal = DIRS[i]
        nb_cell = (cell[0] + normal[0],cell[1] + normal[1])
        if not exists(nb_cell):
            continue
        pot = potential[nb_cell]
        # potential in that neighbour field
        if dir_s == 'right':
            face_index = (nb_cell[0] + 1,nb_cell[1])
        elif dir_s == 'up':
            face_index = (nb_cell[0], nb_cell[1] + 1)
        else:
            face_index = nb_cell
        # Left/right is x, up/down is y
        cost = costs[i%2][face_index]
        # Proposed cost along this direction
        adjacent_potentials[i] = pot + cost
        # If it is cheaper to go from the opposite direction
        if adjacent_potentials[i] < adjacent_potentials[(i+2)%4]:
            pots_from_axis[i%2] = pot
            costs_from_axis[i%2] = cost
        hor_pot,ver_pot = pots_from_axis
        hor_cost,ver_cost = costs_from_axis
        # Coefficients of quadratic equation (upwind discretization)
        a = 1 / hor_cost ** 2 + 1 / ver_cost ** 2
        b = -2 * (hor_pot / hor_cost ** 2 + ver_pot / ver_cost ** 2)
        c = (hor_pot / hor_cost) ** 2 + (ver_pot / ver_cost) ** 2 - 1

        D = b ** 2 - 4 * a * c
        x_high = (2 * c) / (-b - math.sqrt(D))
        # Might not be obvious, but why we take the largest root is found in report.
        return x_high       

# def set_edge_values(u,phi_x,phi_y,val):
#     locs = np.where(u==val)
#     phi_x[locs]=val
#     phi_x[(locs[0],locs[1]+1)] = val
#     phi_y[locs]=val
#     phi_y[(locs[0]+1,locs[1])] = val
#     # No return necessary

def compute_distance_transform(u):
    """
    Compute the weighted distance transform with cost/image function u using a fast marching algorithm
    We compute the distance transform on a staggered grid: this is more precise
    :param u: nonnegative 2D array with cost in each cell/pixel, infinity is allowed.
    :return: weighted distance transform
    
    """
    # nx,ny = u.shape
    
    # Cost for moving along horizontal lines
    u_x = np.ones([nx+1,ny])*np.inf
    u_x[1:-1,:] = (u[1:,:] + u[:-1,:])/2
    # Cost for moving along vertical lines
    u_y = np.ones([nx,ny+1])*np.inf
    u_y[:,1:-1] = (u[:,1:] + u[:,:-1])/2
    
    
    # Initialize locations (known/unknown/exit/obstacle)
    phi = np.ones_like(u)*np.inf
    exit_locs = np.where(u==0)
    obstacle_locs = np.where(u==np.inf)
    phi[exit_locs] = 0
    known_cells = {cell for cell in zip(exit_locs[0],exit_locs[1])}
    unknown_cells = all_cells - known_cells - {cell for cell in zip(obstacle_locs[0],obstacle_locs[1])}
    new_candidate_cells = get_new_candidate_cells(known_cells, unknown_cells)
    candidate_cells = {cell:np.inf for cell in new_candidate_cells}
    while unknown_cells:
        for cell in new_candidate_cells:
            potential = compute_potential(cell,[u_x,u_y],phi)
            candidate_cells[cell] = potential
        sorted_candidates = sorted(candidate_cells.items(), key=operator.itemgetter(1))
        best_cell = sorted_candidates[0][0]
        min_potential = candidate_cells.pop(best_cell)
        phi[best_cell] = min_potential
        unknown_cells.remove(best_cell)
        known_cells.add(best_cell)
        new_candidate_cells = get_new_candidate_cells({best_cell})
            
    # While there are unknown cells:
    # Compute the value of the candidate cells from the known cells
    # Pick the cheapest candidate cell, make it known
    # Get new candidate cells and continue while
    print(phi[obstacle_locs])
    return phi
    

In [ ]:
# Example:
"""
- - - -
* * * -
- * - -
- - - -
"""
nx=ny = 4
cells = np.ones([nx,ny])
cells[1,1] = 0
cells[0:3,2] = 0
cells[3,1:3]=np.inf
print(np.rot90(cells))
p1,p2 = compute_distance_transform(cells)
print(np.rot90(p1))
print(np.rot90(p2))